## OPTIMIZATION AND ANALYTICS
# HOMEWORK 2: Network Optimization and Non-linear Models
#### Óscar Montoya Amores, NIA: 1000451858, Data Science and Engineering

# 1 Network optimization:

### SUMMARY
My Network Optimization problem will be based in the optimization of a Maximum Flow problem.

For this, the problem I've created is based in the current situation the world is going through with the war in Ukraine and the economic problems derived from it.
As maybe you know, Ukraine is one of the Top 5 countries with more volume of wheat, and it's derivatives, exported, with around 20.000.000 of tons exported every year.
Due the situation it's territory is currently going on, this exporations and market is in a bad position. But this is not only a problem for Ukraine, as Ukraninan's wheat is an important product for Europe, African Countries and the Middle East area.

![wheat](trigo_ucrania.jpg "Ukranian Wheat")

With this explained, I can explain the Maximum Flow problem to solve:

As the situation generates uncertainty, Ukranian Government and the countries in need of it's wheat, search to export as much porducts as possible so the wheat doesn't get caught in situations where it can't go out of the country, making Ukraine lose money, and generating needs in the buyers.

This is the reason all of the parties involved want to maximize the flow of wheat out of the country.

And this optimization is the problem I will solve.

The situation is reviewed month by month, so the optimization is focused on maximizing the flow each month.

With the known data, we get that Ukraine exports a mean of 1.6 millons of tons per month. So we will work with 1.600 thousands of tons  per month to distribute.

Of course the <em><strong>channels to use have a maximum capacity</strong></em>, and also <em><strong> the buyers require a minimum per month</strong></em>.

So in addition to maximize the flow, also each buyer require a minimum, they are all in accordance to receive more than what they need and also that one buyer receive more than them or more than what they need, but not less.

![flow](flow.png "Flow")


This is the graph with all the flows and capacities for transporting the wheat, as you see, all comes from Ukraine and have 3 final desinations.

Of course, it goes through a lot of countries with conecctions between them, before arriving at final points. This 3 final points are United Arab Emirates (middle-east), Argelia for the North of Africa, and for Europe there is general distributor.

This 3 buyers, have already give the minimum quantity of wheat they will accept in order to continue with this maximization flow pact of wheat they have done with Ukraine:

$Destination$ | $Minimum$ |
 -------|--- |
UAE | 500 |
 Argelia | 400 |
European Distributor | 600 |


Wiht this last clarification I will start to state and solve the problem, as we have all the data needed for maximizing the flow, and also the 3 constraints the receiving countries have asked for.

### MODEL AND FORMULATION

#### SETS

$C$ = countries (each of them is a node)

$T$ = connections(connections between nodes)and capacity

$M$ = set of minimum quantities of wheat required by the acquisitors

#### PARAMETERS

$initial$ = the point where the flow comes from

$destination$ = the nodes receiving the flow, we have 3

$capacitys$ = each of the capacities of the possible conections, will take a value depending on the connection

$minimums$ = minimums required by the destinations

#### VARIABLES

In this problem, our variables are the connections between countries, the <strong>flow</strong> so we have:

$flow_{id}$ = flow between countries being $i$ the origin or source and $d$ the desitination


#### OBJECTIVE

The objective is that a maximum amount of wheat arrives at the final nodes, so we need to sum the flow of the connections that arrive at one of the final destinations.

$\max \sum_{i,d \in T} flow_{id},  \forall d \in destination$


#### CONSTRAINTS

We have 3 different constraints:

First, we need to limit the flow of wheat in each connection so it doesn't surpass the capacity:
        
$\forall{i,d \in T}, flow_{id} \leq capacity_{id}$
        
        
The second is a rule of the maximization flow problems, all the incomming flow, should also go out, except if we are in the source or the end where flow only go out or comes in respectively:
        
$\forall{c \in C}, \sum_{i \in T} flow_{ic}$==$ \sum_{d \in T} flow_{cd}$ BUT ONLY IF $c$!=$d \in destination$ or $i \in initial$

        
Lastly, due to the pact of the buyers, ensuring a minimum amount of wheat for each of them is mandatory:
        
$\sum_{j \in minimums}\sum_{s \in S} flow_{sj} \geq minimums_j {j \in minimums}$ For each final destination, the sum of all the flows that arrive at it, must be greater or equal than the minimum of such final point.

</ul>

### CODE

#### ENTER THE DATA

In [1]:
%%writefile maxflow.dat
set C := Ukraine Turkey Romania Poland Israel Greece Germany SaudiArabia Egypt UAE Argelia EUDistributor;  #Set of nodes, each country is one
set T := (Ukraine,Turkey) (Ukraine,Romania) (Ukraine,Poland) (Turkey,Israel) (Turkey,Greece) (Romania,Turkey) (Romania,Greece) (Poland,Germany) (Israel,UAE) (Israel,SaudiArabia) (Greece,Israel) (Greece,Egypt) (Greece,EUDistributor) (Germany,EUDistributor) (SaudiArabia,UAE) (SaudiArabia, Egypt) (Egypt,Argelia);  #All the connections
#set M := UAE Argelia EUDistributor;
#capacity of the nodes
param initial := Ukraine;
param: destinations := 
UAE 
Argelia 
EUDistributor;
param: capacity :=
Ukraine Turkey 800
Ukraine Romania 400
Ukraine Poland 400
Turkey Israel 600
Turkey Greece 500
Romania Turkey 300
Romania Greece 300
Poland Germany 400
Israel UAE 400 
Israel SaudiArabia 600 
Greece Israel 400
Greece Egypt 300
Greece EUDistributor 300
Germany EUDistributor 800
SaudiArabia UAE 450
SaudiArabia Egypt 200
Egypt Argelia 500;
param: minimums:= 
UAE 500 
Argelia 400 
EUDistributor 600;

Overwriting maxflow.dat


#### MODEL IN CODE

In [2]:
%%writefile maxflow.py

from pyomo.environ import *

model= AbstractModel()

#NODES (countries)
model.C = Set()
#ARCS BETWEEN NODES
model.T = Set(within = model.C*model.C) #not more than one arc between nodes

#SOURCE
model.initial = Param(within = model.C)
#So we have a inital point called initial and its destination will be destination
#CAPACITIES OF EACH ARC
model.capacity = Param(model.T) # we cannot exceed these flow capacity limits
#MINIMUMS OF THE DESTINATIONS
model.minimums = Param(model.C)
#FINAL DESTINATIONS
model.destinations=Param(model.C)

#THE DECISION VARIABLE IS THE FLOWid
model.flow = Var(model.T ,within=NonNegativeReals) #model.T as is the one with all the connections, so number of connections is equal to the number of variables

#OBJECTIVE FUNCTION
#Maximize the flow so the final destinations receive as much as possible
def objective(model):
    total=0
    for (i,d) in model.T:
            if d=="UAE" or d=="Argelia" or d=="EUDistributor": #FINAL DESTINATIONS
                total+=model.flow[i,d]
    return total
    
model.obj = Objective(rule=objective, sense=maximize) #we are looking to maximize


#CONSTRAINTS
#1)Limit the flow to the capacity of the connection
def capacity_limit(model, i, d):
    return model.flow[i,d] <= model.capacity[i, d] #the flow between i (source node) and (d) destination, should be less than the pre-defined capacity between i and d


#2)Rule of the maximization problems, all the flow coming to a node, should also go out, except if we are in the first, source node, that it only goes out, 
#or if we are in one of the final destinations, that only receives the flow.
def total_flow_rule(model, k):
    if value(model.initial) == k: #if the node we are is the initial one, or one of the final destinations, skip
        return Constraint.Skip
    if k=="UAE" or k=="Argelia" or k=="EUDistributor": #FINAL DESTINATIONS
            return Constraint.Skip
    inFlow = sum(model.flow[i,d] for (i,d) in model.T if d==k) #flow entering the node
    outFlow = sum(model.flow[i,d] for (i,d) in model.T if i==k) #flow exiting the node
    return inFlow == outFlow #both should be equal


    
#3)MAKE SURE THE FINAL DESTINATIONS RECEIVE THE MINIMUM AMOUNT OF WHEAT PACTED FOR EACH OF THEM
def min_wheat(model, k):
    if k=="UAE" or k=="Argelia" or k=="EUDistributor": #FINAL DESTINATIONS
            return sum(model.flow[i, d] for (i,d) in model.T if d==k)>=model.minimums[k]
    else:
        return Constraint.Skip
        
#INCLUDE THEM IN THE MODEL
model.capacity_limit = Constraint(model.T, rule = capacity_limit)

model.total_flow_rule = Constraint(model.C, rule = total_flow_rule)

model.min_wheat = Constraint(model.C, rule = min_wheat)

Overwriting maxflow.py


#### GET THE SOLUTION

In [3]:
!pyomo solve --solver=glpk maxflow.py maxflow.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
    'Any'. The default domain for Param objects is 'Any'.  However, we will be
    changing that default to 'Reals' in the future.  If you really intend the
    domain of this Paramto be 'Any', you can suppress this warning by
    explicitly specifying 'within=Any' to the Param constructor.  (deprecated
    in 5.6.9, will be removed in (or after) 6.0) (called from
    C:\Users\34649\anaconda3\lib\site-
    packages\pyomo\core\base\PyomoModel.py:825)
[    0.16] Applying solver
[    0.25] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 1600.0
    Solver results file: results.yml
[    0.25] Applying Pyomo postprocessing actions
[    0.25] Pyomo Finished


errorcode: 0
retval:
    instance: <pyomo.core.base.PyomoModel.ConcreteModel object at 0x000002ACFFD1B720>
    local:
        time_initial_import: 0.003503084182739258
        usermodel: <module 'maxflow' from 'C:\\Users\\34649\\Desktop\\UNIVERSIDAD\\TERCERO\\OPTIMIZATION AND ANALYTICS\\HW2\\maxflow.py'>
    options: <pyomo.common.config.ConfigDict object at 0x000002AC84FD7A40>
    results: {'Problem': [{'Name': 'unknown', 'Lower bound': 1600.0, 'Upper bound': 1600.0, 'Number of objectives': 1, 'Number of constraints': 29, 'Number of variables': 18, 'Number of nonzeros': 49, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.04059767723083496}], 'Solution': [OrderedDict([('number of solutions', 1), ('number of solutions displayed', 1)]), {'Gap': 0.0, 'Status': 'feasible', 'Message': None, 'Problem': {}, 'Objective': {

In [4]:
import yaml
with open('results.yml') as f:
    doc = yaml.load(f,Loader=yaml.FullLoader)
    l0 = doc["Problem"]
    l1 = doc["Solution"][1]["Variable"]
    l1 = list(l1.items())
    
print("Number of objectives",l0[0]["Number of objectives"])
print("Lower bound",l0[0]["Lower bound"],"= Upper bound",l0[0]["Upper bound"], "---->OBJECTIVE FUNCTION MAXIMIZED")
print("Number of constraints",l0[0]["Number of constraints"])
print("Number of variables",l0[0]["Number of variables"])  


# Print flows used for maximize:  
print("\nFlows used:")
for i in l1:
    print(i[0].replace("flow", ""))
    print(i[1]["Value"])


Number of objectives 1
Lower bound 1600.0 = Upper bound 1600.0 ---->OBJECTIVE FUNCTION MAXIMIZED
Number of constraints 29
Number of variables 18

Flows used:
[Egypt,Argelia]
400
[Germany,EUDistributor]
400
[Greece,EUDistributor]
300
[Greece,Egypt]
300
[Israel,SaudiArabia]
200
[Israel,UAE]
400
[Poland,Germany]
400
[Romania,Greece]
300
[Romania,Turkey]
100
[SaudiArabia,Egypt]
100
[SaudiArabia,UAE]
100
[Turkey,Greece]
300
[Turkey,Israel]
600
[Ukraine,Poland]
400
[Ukraine,Romania]
400
[Ukraine,Turkey]
800


#### INTERPRETATION OF THE RESULTS

Above you can see the nodes the maximization has used to give the final destinations the maximum possible flow.

Going in order, the problem has recognized well the objective function. We can see there is only one objective and that the upper bound and the lower bound coincide in the same number 1600. Why I know this is well solved? Checking the graph of the flows you can see that the flow that goes out of Ukraine is less than the flow that the 3 final destinations can receive together, so the total flow should be maximized to the value going out of the first source. 

In addition we can also see that the number of variables correspond to all of the possible connections and that we have 29 constraints that the optimization needs to accomplish.

With respect the solution we get above, we can see the connections that give the maximum flow at the end nodes, respecting the constraints. The ones that are not showed are just 0's, they are not used or they are not necessary as there are better options.

They key point of this problem has been already commented above, the countries don't have a problem in receiving more than what they need, maximizing the flow. But also, the tricky point is the fact that the initial flow is less than what the final nodes and connections can manage to take. And that's why the 3 destinations, Algeria, UAE and EUDistributor, asked for a minimum amount of wheat each one, protecting in this way themselves of the flow being maximized without one of them, as the sum of the three of them results in a maximum capacity of 2450 but we have a flow of 1600 only.

Now, we can check the results. First of all, the total flow rule, that asked for the in-Flow to be the same that the out-Flow, is clearly respected, as the initial flow is 1600 and the final one is also 1600 so we don't lose flow in any node or connection. And the capacities are all also respected.

So finally, let's comment the last steps to each of the final destinations, to see if their minimums are fulfilled:

<ul>
    <li>UAE, Minimum: 500. We can see Israel is sending 400 thousands of tons to UAE their maximum capacity, and Saudi Arabia another 100, which results in 500 thousands of tons, exactly the minimum required by this country.
    <li>Argelia, Minimum: 400. Is receiving 400 from Egypt, again, respecting the minimum amount pacted.
    <li>EUDistributor, Minimum:600. Germany is sending 400 and another 300 comes from Greece, making 700 thousand of tons, 100 more than the required.
    <li> The total sum is 1600 thousands of tons of wheat, the maximum quantity Ukraine can get off the country, so the objective is done, and the countries will be able to continue buying to Ukraine, and not lose wheat for their own countries, and Ukraine will still get benefits from the wheat.
</ul>
To sum up, I can confirm all the constraints are respected and the objective fulfilled, so no country will lose anything in the uncertain situation with respect the wheat and Ukraine.